In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pympc.models.boxatlas_parametric import BoxAtlas, MovingLimb, FixedLimb, Trajectory
from pympc.approximated_controller_full_feasible_sets import ApproximatedHybridModelPredictiveController, PolicySampler, upload_PolicySampler
from pympc.geometry.polytope import Polytope
import pympc.plot as mpc_plt

# Construction of Box-Atlas

Robot's limbs

In [3]:
from pympc.models.boxatlas_parameters import dynamics
penetration = dynamics['mass'] * dynamics['gravity'] / dynamics['normal_stiffness']

# right hand
A_rh = {
    'not_in_contact': np.array([[-1., 0.],[0., -1.]]),
    'in_contact': np.array([[1., 0.],[-1., 0.],[0., -1.]]),
}
b_rh = {
    'not_in_contact': np.array([[.4],[.5]]),
    'in_contact': np.array([[-.4],[.4+penetration],[.5]]),
}
contact_surfaces_rh = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_rh = np.array([[-.3], [.0]])
parametric_wall_distance = {
    'label': 'wall_distance_rh',
    'not_in_contact': {'index': [0], 'coefficient': [1.]},
    'in_contact': {'index': [0, 1], 'coefficient': [-1., 1.]},
    'min': -.05,
    'max': .05,
} # if positive moves the wall away from the body
parameters_rh = [parametric_wall_distance]
rh = MovingLimb(A_rh, b_rh, contact_surfaces_rh, q_rh, parameters=parameters_rh)

# left foot
A_lf = {
    'not_in_contact': np.array([[0., -1.]]),
    'in_contact': np.array([[0., 1.],[0., -1.]]),
}
b_lf = {
    'not_in_contact': np.array([[.5]]),
    'in_contact': np.array([[-.5],[.5+2.*penetration]]),
}
contact_surfaces_lf = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_lf = np.array([[.2], [-.5 - penetration / 2.]])
lf = MovingLimb(A_lf, b_lf, contact_surfaces_lf, q_lf)

# right foot
A_rf = {
    'not_in_contact': np.array([[-1., 0.],[0., -1.]]),
    'in_contact': np.array([[0., 1.],[-1., 0.],[0., -1.]]),
}
b_rf = {
    'not_in_contact': np.array([[.5],[.5]]),
    'in_contact': np.array([[-.5],[.5],[.5+2.*penetration]]),
}
contact_surfaces_rf = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_rf = np.array([[-.2], [-.5 - penetration / 2.]])
rf = MovingLimb(A_rf, b_rf, contact_surfaces_rf, q_rf)

Assemble robot

In [4]:
limbs = { 'moving': {'rh': rh, 'lf': lf, 'rf': rf}, 'fixed': {} }
nominal_mode = {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}
forbidden_modes = [{'lf': 'not_in_contact', 'rf': 'not_in_contact'}]
box_atlas = BoxAtlas(limbs, nominal_mode, forbidden_modes)

Computation of Maximal Invariant Constraint-Admissible Set started.
Determinedness index: 6, Convergence index: 0.0, Number of facets: 231.                              
Maximal Invariant Constraint-Admissible Set found.
Removing redundant facets ... minimal facets are 77.
Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Parameter FeasibilityTol unchanged
   Value: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Parameter NumericFocus unchanged
   Value: 0  Min: 0  Max: 3  Default: 0
Parameter OptimalityTol unchanged
   Value: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Parameter MIPGap unchanged
   Value: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Parameter IntFeasTol unchanged
   Value: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05
Parameter MIPFocus unchanged
   Value: 0  Min: 0  Max: 3  Default: 0


Print state, input, and modes

In [5]:
box_atlas.print_state_labels()
box_atlas.print_input_labels()
box_atlas.print_modes()


Box-Atlas states:
0: wall_distance_rh
1: qlfx
2: qlfy
3: qrfx
4: qrfy
5: qrhx
6: qrhy
7: qbx
8: qby
9: tb
10: vbx
11: vby
12: ob

Box-Atlas inputs:
0: vlfx
1: vlfy
2: vrfx
3: vrfy
4: vrhx
5: vrhy

Box-atlas modes:
0: {'lf': 'not_in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}
1: {'lf': 'not_in_contact', 'rf': 'in_contact', 'rh': 'in_contact'}
2: {'lf': 'in_contact', 'rf': 'not_in_contact', 'rh': 'not_in_contact'}
3: {'lf': 'in_contact', 'rf': 'not_in_contact', 'rh': 'in_contact'}
4: {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'not_in_contact'}
5: {'lf': 'in_contact', 'rf': 'in_contact', 'rh': 'in_contact'}


# Hybrid MPC

In [ ]:
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = 0.
x_0[10,0] = -1.5
u, x, ss = box_atlas.controller.feedforward(x_0)[0:3]

In [ ]:
prog = box_atlas.controller.condense_program(ss)
print prog.C_u.shape

Closed-loop system simulation of the MPC controller

In [ ]:
N_sim = 50
u = []
# ['wall_distance_rh', 'qlfx', 'qlfy', 'qrfx', 'qrfy', 'qrhx', 'qrhy', 'qbx', 'qby', 'tb', 'vbx', 'vby', 'ob']
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[1,0] = .15
x_0[3,0] = .15
x = [x_0]
u_ws = None
x_ws = None
ss_ws = None
nominal_mode = box_atlas.get_mode_index(box_atlas.nominal_mode)
for k in range(N_sim):
    u_k, x_k, ss_k = box_atlas.controller.feedforward(x[k], u_ws, x_ws, ss_ws)[0:3]
    print 'Time step ' + str(k) + ': ' + str(ss_k) + '.'
    u_ws = u_k[1:] + [box_atlas.K.dot(x_k[-1])]
    x_ws = x_k[1:] + [box_atlas.pwa_system.simulate(x_k[-1], [u_ws[-1]])[0][1]]
    ss_ws = ss_k[1:] + (nominal_mode,)
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k[0]])[0][1]
    # print 'Planner-simulator missmatch:', np.linalg.norm(x_next - x_k[1])
    u.append(u_k[0])
    x.append(x_next)
# traj_miqp = Trajectory(x, u, box_atlas.Q, box_atlas.R, box_atlas.P)
# np.save('box_atlas_tajectory_miqp_lh', traj_miqp)

Visualization of the simulation with Director

In [ ]:
from pympc.models.boxatlas_parameters import dynamics
box_atlas.visualize(x[0])
time.sleep(10 * dynamics['sampling_time'])
for x_t in x:
    box_atlas.visualize(x_t)
    time.sleep(dynamics['sampling_time'])

In [ ]:
box_atlas.print_mode_sequence(ss)

# Approximate Hybrid MPC

Initialization of the library

In [6]:
sampler = PolicySampler(box_atlas.controller)
samples_so_far = 14600
if samples_so_far > 0:
    sampler = upload_PolicySampler('sampler_box_atlas_' + str(samples_so_far))

Parameter FeasibilityTol unchanged
   Value: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06
Parameter NumericFocus unchanged
   Value: 0  Min: 0  Max: 3  Default: 0
Parameter OptimalityTol unchanged
   Value: 1e-06  Min: 1e-09  Max: 0.01  Default: 1e-06


Coverage

In [ ]:
sample_step = 100
n_steps = 100
for i in range(n_steps):
    n = samples_so_far + (i+1)*sample_step
    sampler.sample_policy(sample_step, box_atlas.is_inside_a_domain)
    print(str(n) + ' samples added to the library.')
    sampler.save('sampler_box_atlas_' + str(n))

Synthesize approximated contoller

In [7]:
approximated_controller = ApproximatedHybridModelPredictiveController(sampler, box_atlas.get_mode_index(nominal_mode))

In [ ]:
approximated_controller.tree.plot('box_atlas_tree')

In [8]:
len(sampler.qp_library)

98

In [ ]:
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = 0.
x_0[10,0] = -1.5
u, ms = approximated_controller.feedback(x_0)

In [ ]:
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = 0.
x_0[10,0] = -1.5
u, x, ss = box_atlas.controller.feedforward(x_0)[0:3]

Closed-loop system simulation of the approximate MPC controller

In [9]:
N_sim = 50
u = []
# ['wall_distance_rh', 'qlfx', 'qlfy', 'qrfx', 'qrfy', 'qrhx', 'qrhy', 'qbx', 'qby', 'tb', 'vbx', 'vby', 'ob']
x_0 = np.zeros((box_atlas.n_x, 1))
x_0[0,0] = 0.
x_0[10,0] = -1.5
x = [x_0]
ms_ws = None
for k in range(N_sim):
    print '\nTime step ' + str(k) + ':'
    u_k, ms_ws = approximated_controller.feedback(x[k], ms_ws)
    ms_ws = ms_ws[1:] + (box_atlas.get_mode_index(nominal_mode),)
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k])[0][1]
    u.append(u_k)
    x.append(x_next)


Time step 0:
No warm start provided.
Checking mode sequence fragment (5,):
    infeasible, branch cut.
Checking mode sequence fragment (4,):
    further branch.
Checking mode sequence fragment (4, 3, 1, 5, 5, 3, 3, 3, 3, 5):
    new upper bound with cost 4.53882128709.
Checking mode sequence fragment (4, 5, 5, 3, 3, 3, 4, 4, 0, 0):
    suboptimal (cost is 4.56458112464), branch cut.
Checking mode sequence fragment (4, 0):
    further branch.
Checking mode sequence fragment (4, 0, 4, 2, 2, 2, 2, 0, 0, 0):
    infeasible, branch cut.
Checking mode sequence fragment (4, 0, 2, 5, 3, 3, 3, 3, 3, 5):
    suboptimal (cost is 5.88310303069), branch cut.
Checking mode sequence fragment (4, 0, 1, 1, 0, 4, 3, 3, 3, 3):
    suboptimal (cost is 4.82298571503), branch cut.
Checking mode sequence fragment (4, 1):
    further branch.
Checking mode sequence fragment (4, 1, 0, 0, 0, 0, 4, 2, 3, 3):
    suboptimal (cost is 4.7185185624), branch cut.
Checking mode sequence fragment (4, 1, 1, 4, 3, 3, 3, 

    suboptimal (cost is 0.936515141967), branch cut.
Checking mode sequence fragment (4, 5, 5, 3, 3, 3, 4, 4, 0, 0):
    suboptimal (cost is 0.666051486249), branch cut.
Checking mode sequence fragment (4, 0):
    further branch.
Checking mode sequence fragment (4, 0, 4, 2, 2, 2, 2, 0, 0, 0):
    suboptimal (cost is 1.62316068759), branch cut.
Checking mode sequence fragment (4, 0, 2, 5, 3, 3, 3, 3, 3, 5):
    suboptimal (cost is 0.882805243955), branch cut.
Checking mode sequence fragment (4, 0, 1, 1, 0, 4, 3, 3, 3, 3):
    infeasible, branch cut.
Checking mode sequence fragment (4, 1):
    further branch.
Checking mode sequence fragment (4, 1, 0, 0, 0, 0, 4, 2, 3, 3):
    infeasible, branch cut.
Checking mode sequence fragment (4, 1, 1, 4, 3, 3, 3, 3, 3, 3):
    suboptimal (cost is 1.12595885176), branch cut.
Checking mode sequence fragment (4, 1, 2, 3, 3, 3, 3, 4, 0, 0):
    suboptimal (cost is 1.31725105365), branch cut.
Checking mode sequence fragment (4, 2):
    further branch.
C

Checking mode sequence fragment (3, 3, 3, 3, 3, 3, 3, 3, 4, 0):
    suboptimal (cost is 0.421950002499), branch cut.
Checking mode sequence fragment (3, 3, 3, 3, 3, 2, 4, 0, 0, 0):
    suboptimal (cost is 0.801945580318), branch cut.
Checking mode sequence fragment (3, 2):
    further branch.
Checking mode sequence fragment (3, 2, 2, 4, 0, 0, 0, 4, 4, 4):
    suboptimal (cost is 0.902874188807), branch cut.
Checking mode sequence fragment (3, 2, 0, 2, 2, 2, 2, 4, 0, 0):
    suboptimal (cost is 2.95553631053), branch cut.
Checking mode sequence fragment (2,):
    infeasible, branch cut.
Checking mode sequence fragment (1,):
    infeasible, branch cut.
Checking mode sequence fragment (0,):
    infeasible, branch cut.
Found solution with cost 0.137670436656 and mode sequence (3, 3, 3, 3, 4, 4, 4, 4, 4, 4).

Time step 7:
Loaded solution with cost 0.11901242776 and mode sequence (3, 3, 3, 4, 4, 4, 4, 4, 4, 4).
Checking mode sequence fragment (5,):
    further branch.
Checking mode sequence 

    suboptimal (cost is 1.84174236622), branch cut.
Checking mode sequence fragment (5, 5, 3, 3, 3, 3, 3, 3, 3, 0):
    suboptimal (cost is 0.568817195015), branch cut.
Checking mode sequence fragment (5, 4, 2, 2, 2, 4, 4, 4, 0, 4):
    suboptimal (cost is 1.22904362302), branch cut.
Checking mode sequence fragment (5, 2):
    suboptimal (cost is 0.0824229876622), branch cut.
Checking mode sequence fragment (5, 1):
    suboptimal (cost is 0.292086013611), branch cut.
Mode sequence fragment (4,) not checked:
    further branch.
Checking mode sequence fragment (4, 3, 1, 5, 5, 3, 3, 3, 3, 5):
    suboptimal (cost is 0.740406566224), branch cut.
Checking mode sequence fragment (4, 5, 5, 3, 3, 3, 4, 4, 0, 0):
    suboptimal (cost is 0.502169878075), branch cut.
Checking mode sequence fragment (4, 0):
    suboptimal (cost is 0.292032690837), branch cut.
Checking mode sequence fragment (4, 1):
    suboptimal (cost is 0.292085861532), branch cut.
Checking mode sequence fragment (4, 2):
    sub

Mode sequence fragment (4,) not checked:
    further branch.
Checking mode sequence fragment (4, 3, 1, 5, 5, 3, 3, 3, 3, 5):
    suboptimal (cost is 0.839178319324), branch cut.
Checking mode sequence fragment (4, 5, 5, 3, 3, 3, 4, 4, 0, 0):
    suboptimal (cost is 0.543687005426), branch cut.
Checking mode sequence fragment (4, 0):
    suboptimal (cost is 0.145970516693), branch cut.
Checking mode sequence fragment (4, 1):
    suboptimal (cost is 0.183369485027), branch cut.
Checking mode sequence fragment (4, 2):
    suboptimal (cost is 0.141308807774), branch cut.
Mode sequence fragment (4, 4) not checked:
    further branch.
Checking mode sequence fragment (4, 4, 4, 0, 0, 4, 4, 2, 2, 4):
    suboptimal (cost is 0.712522134674), branch cut.
Checking mode sequence fragment (4, 4, 2, 2, 2, 2):
    suboptimal (cost is 0.870495551517), branch cut.
Checking mode sequence fragment (3,):
    infeasible, branch cut.
Checking mode sequence fragment (2,):
    infeasible, branch cut.
Checking 

    suboptimal (cost is 0.595485799817), branch cut.
Checking mode sequence fragment (4, 0):
    suboptimal (cost is 0.144859673892), branch cut.
Checking mode sequence fragment (4, 1):
    suboptimal (cost is 0.236945493454), branch cut.
Checking mode sequence fragment (4, 2):
    suboptimal (cost is 0.137502683269), branch cut.
Mode sequence fragment (4, 4) not checked:
    further branch.
Checking mode sequence fragment (4, 4, 4, 0, 0, 4, 4, 2, 2, 4):
    suboptimal (cost is 0.702427840615), branch cut.
Checking mode sequence fragment (4, 4, 2, 2, 2, 2):
    suboptimal (cost is 0.882970442288), branch cut.
Checking mode sequence fragment (3,):
    infeasible, branch cut.
Checking mode sequence fragment (2,):
    infeasible, branch cut.
Checking mode sequence fragment (1,):
    infeasible, branch cut.
Checking mode sequence fragment (0,):
    infeasible, branch cut.
Found solution with cost 0.00301503969743 and mode sequence (4, 4, 4, 4, 4, 4, 4, 4, 4, 4).

Time step 23:
Loaded solut

    suboptimal (cost is 0.699597840589), branch cut.
Checking mode sequence fragment (4, 4, 2, 2, 2, 2):
    suboptimal (cost is 0.889602402179), branch cut.
Checking mode sequence fragment (3,):
    infeasible, branch cut.
Checking mode sequence fragment (2,):
    infeasible, branch cut.
Checking mode sequence fragment (1,):
    infeasible, branch cut.
Checking mode sequence fragment (0,):
    infeasible, branch cut.
Found solution with cost 0.000924427166521 and mode sequence (4, 4, 4, 4, 4, 4, 4, 4, 4, 4).

Time step 29:
Loaded solution with cost 0.000760936815251 and mode sequence (4, 4, 4, 4, 4, 4, 4, 4, 4, 4).
Checking mode sequence fragment (5,):
    infeasible, branch cut.
Mode sequence fragment (4,) not checked:
    further branch.
Checking mode sequence fragment (4, 3, 1, 5, 5, 3, 3, 3, 3, 5):
    suboptimal (cost is 0.94598355618), branch cut.
Checking mode sequence fragment (4, 5, 5, 3, 3, 3, 4, 4, 0, 0):
    suboptimal (cost is 0.644648395732), branch cut.
Checking mode se

    suboptimal (cost is 0.69861307668), branch cut.
Checking mode sequence fragment (4, 4, 2, 2, 2, 2):
    suboptimal (cost is 0.893113124947), branch cut.
Checking mode sequence fragment (3,):
    infeasible, branch cut.
Checking mode sequence fragment (2,):
    infeasible, branch cut.
Checking mode sequence fragment (1,):
    infeasible, branch cut.
Checking mode sequence fragment (0,):
    infeasible, branch cut.
Found solution with cost 0.000288725352028 and mode sequence (4, 4, 4, 4, 4, 4, 4, 4, 4, 4).

Time step 35:
Loaded solution with cost 0.000238008375722 and mode sequence (4, 4, 4, 4, 4, 4, 4, 4, 4, 4).
Checking mode sequence fragment (5,):
    infeasible, branch cut.
Mode sequence fragment (4,) not checked:
    further branch.
Checking mode sequence fragment (4, 3, 1, 5, 5, 3, 3, 3, 3, 5):
    suboptimal (cost is 0.9736398384), branch cut.
Checking mode sequence fragment (4, 5, 5, 3, 3, 3, 4, 4, 0, 0):
    suboptimal (cost is 0.671255646312), branch cut.
Checking mode sequ

    further branch.
Checking mode sequence fragment (4, 4, 4, 0, 0, 4, 4, 2, 2, 4):
    suboptimal (cost is 0.698245883072), branch cut.
Checking mode sequence fragment (4, 4, 2, 2, 2, 2):
    suboptimal (cost is 0.895001530712), branch cut.
Checking mode sequence fragment (3,):
    infeasible, branch cut.
Checking mode sequence fragment (2,):
    infeasible, branch cut.
Checking mode sequence fragment (1,):
    infeasible, branch cut.
Checking mode sequence fragment (0,):
    infeasible, branch cut.
Found solution with cost 9.07784595751e-05 and mode sequence (4, 4, 4, 4, 4, 4, 4, 4, 4, 4).

Time step 41:
Loaded solution with cost 7.48826403434e-05 and mode sequence (4, 4, 4, 4, 4, 4, 4, 4, 4, 4).
Checking mode sequence fragment (5,):
    infeasible, branch cut.
Mode sequence fragment (4,) not checked:
    further branch.
Checking mode sequence fragment (4, 3, 1, 5, 5, 3, 3, 3, 3, 5):
    suboptimal (cost is 0.990342835362), branch cut.
Checking mode sequence fragment (4, 5, 5, 3, 3, 

    suboptimal (cost is 0.69809833545), branch cut.
Checking mode sequence fragment (4, 4, 2, 2, 2, 2):
    suboptimal (cost is 0.896022052774), branch cut.
Checking mode sequence fragment (3,):
    infeasible, branch cut.
Checking mode sequence fragment (2,):
    infeasible, branch cut.
Checking mode sequence fragment (1,):
    infeasible, branch cut.
Checking mode sequence fragment (0,):
    infeasible, branch cut.
Found solution with cost 2.86243521575e-05 and mode sequence (4, 4, 4, 4, 4, 4, 4, 4, 4, 4).

Time step 47:
Loaded solution with cost 2.36189819374e-05 and mode sequence (4, 4, 4, 4, 4, 4, 4, 4, 4, 4).
Checking mode sequence fragment (5,):
    infeasible, branch cut.
Mode sequence fragment (4,) not checked:
    further branch.
Checking mode sequence fragment (4, 3, 1, 5, 5, 3, 3, 3, 3, 5):
    suboptimal (cost is 1.00011181624), branch cut.
Checking mode sequence fragment (4, 5, 5, 3, 3, 3, 4, 4, 0, 0):
    suboptimal (cost is 0.696867014528), branch cut.
Checking mode seq

In [ ]:
from pympc.models.boxatlas_parameters import dynamics
box_atlas.visualize(x[0])
time.sleep(10 * dynamics['sampling_time'])
for x_t in x:
    box_atlas.visualize(x_t)
    time.sleep(dynamics['sampling_time'])